In [1]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("worst team I have ever seen play, Sack the manager", top_k = 3)

[{'label': 'negative', 'score': 0.939471960067749},
 {'label': 'neutral', 'score': 0.042314887046813965},
 {'label': 'positive', 'score': 0.01821320690214634}]

In [588]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def make_table_pct(df, week):
    outcomes = pd.DataFrame(columns = ['Team','Positive','Neutral','Negative','Points', 'Week'])
    for team in df['Team'].unique():
        tweet_count = 0
        pred = {'positive':0,'neutral':0,'negative':0} #reset with each team
        tweets = df.loc[df['Team'] == team]['Tweet'] #get relevant tweets
        if df.loc[df['Team'] == team].reset_index(drop = True)['Win'][0] == 1:
            points = 3
        if df.loc[df['Team'] == team].reset_index(drop = True)['Draw'][0] == 1:
            points = 1
        if df.loc[df['Team'] == team].reset_index(drop = True)['Loss'][0] == 1:
            points = 0
        for tweet in tweets:
            preds = sentiment_task(tweet, top_k = 3)
            pred[preds[0]['label']] += preds[0]['score']#get the info
            pred[preds[1]['label']] += preds[1]['score']#get the info
            pred[preds[2]['label']] += preds[2]['score']#get the info
            tweet_count += 1
        outcomes = outcomes.append({'Team' : team[1:],
                         'Positive' : pred['positive']/tweet_count,
                         'Neutral' : pred['neutral']/tweet_count,
                         'Negative' : pred['negative']/tweet_count,
                         'Points' : points,
                         'Week' : week},
                        ignore_index = True)
    return(outcomes)

def make_table_ttls(df, week):
    outcomes = pd.DataFrame(columns = ['Team','Positive','Neutral','Negative','Points', 'Week'])
    for team in df['Team'].unique():
        tweet_count = 0
        pred = {'positive':0,'neutral':0,'negative':0} #reset with each team
        tweets = df.loc[df['Team'] == team]['Tweet'] #get relevant tweets
        if df.loc[df['Team'] == team].reset_index(drop = True)['Win'][0] == 1:
            points = 3
        if df.loc[df['Team'] == team].reset_index(drop = True)['Draw'][0] == 1:
            points = 1
        if df.loc[df['Team'] == team].reset_index(drop = True)['Loss'][0] == 1:
            points = 0
        for tweet in tweets:
            preds = sentiment_task(tweet)
            pred[preds[0]['label']] += 1
            tweet_count += 1
        outcomes = outcomes.append({'Team' : team[1:],
                         'Positive' : pred['positive']/tweet_count,
                         'Neutral' : pred['neutral']/tweet_count,
                         'Negative' : pred['negative']/tweet_count,
                         'Points' : points,
                         'Week' : week},
                        ignore_index = True)
    return(outcomes)

In [591]:
for matchweek in os.listdir('Proj_Data\\data\\Matchweeks'): #where the data is stored
    if matchweek == '.ipynb_checkpoints': #only want the data
        continue
    df = pd.read_csv('Proj_Data\\data\\Matchweeks\\' + matchweek)
    df = df.iloc[:,1:6].copy()
    week = int(matchweek[0])-1 #only have data 2-9
    out_pct = make_table_pct(df, week)
    if int(matchweek[0])-1 == 1: #no need to concat on the first iteration
        final_pct = out_pct.copy()
        continue
    final_pct = pd.concat([final_pct,out_pct]).reset_index(drop = True)

In [592]:
for matchweek in os.listdir('Proj_Data\\data\\Matchweeks'): #where the data is stored
    if matchweek == '.ipynb_checkpoints': #only want the data
        continue
    df = pd.read_csv('Proj_Data\\data\\Matchweeks\\' + matchweek)
    df = df.iloc[:,1:6].copy()
    week = int(matchweek[0])-1
    out_ttl = make_table_ttls(df, week)
    if int(matchweek[0])-1 == 1:
        final_ttl = out_ttl.copy()
        continue
    final_ttl = pd.concat([final_ttl,out_ttl]).reset_index(drop = True)

In [627]:
df = pd.read_csv('Proj_Data\\data\\Matchweeks\\9_Matchweek_32')
df = df.iloc[:,1:6].copy()
week = int(matchweek[0])-1 #only have data 2-9
out_pct = make_table_pct(df, week)
if int(matchweek[0])-1 == 1: #no need to concat on the first iteration
    final_pct = out_pct.copy()
final_pct = pd.concat([final_pct,out_pct]).reset_index(drop = True)

SyntaxError: 'continue' not properly in loop (1940419049.py, line 7)

In [632]:
final_pct.groupby('Team').sum()

,Positive,Neutral,Negative,prediction_points
Team,,,,
Arsenal,1.387896,1.023129,1.588975,9
BrentfordFC,2.927660,2.457791,2.614550,13
CPFC,2.111676,2.164335,2.723990,8
ChelseaFC,1.667835,1.853068,2.479097,1
LUFC,2.457175,1.558493,2.984333,9
ManCity,1.262653,1.326370,1.410977,4
NFFC,2.197667,2.100454,2.701879,7
NUFC,2.728436,1.734772,2.536791,15
OfficialBHAFC,2.416933,1.945253,1.637814,13


In [594]:
final_ttl

,Team,Positive,Neutral,Negative,Points,Week
0,manutd,0.250000,0.250000,0.500000,1,1
1,LUFC,0.254902,0.254902,0.490196,1,1
2,BrentfordFC,0.428571,0.285714,0.285714,0,1
3,everton,0.625000,0.125000,0.250000,3,1
4,fulhamfc,0.500000,0.200000,0.300000,0,1
...,...,...,...,...,...,...
119,lcfc,0.529412,0.235294,0.235294,3,8
120,NFFC,0.456522,0.239130,0.304348,0,8
121,CPFC,0.125000,0.750000,0.125000,1,8
122,SpursOfficial,0.111111,0.222222,0.666667,0,8


In [613]:
def evaluate(df):
    correct = 0
    counter = 0
    prediction = []
    prediction_results = []
    for i in range(len(df)):
        row = df.iloc[i]
        vals = np.array([row['Positive'],row['Neutral'],row['Negative']])
        most = vals.argmax()
        if most == 0:
            prediction.append('Win')
            prediction_results.append(3)
            if row['Points'] == 3:
                correct += 1
        elif most == 1:
            prediction.append('Draw')
            prediction_results.append(1)
            if row['Points'] == 1:
                correct += 1
        elif most == 2:
            prediction.append('Loss')
            prediction_results.append(0)
            if row['Points'] == 0:
                correct += 1
        counter += 1
    return(correct/counter,prediction,prediction_results)

In [615]:
score,prediction_class, prediction_points = evaluate(final_pct)

In [622]:
final_pct['prediction_class'] = prediction_class
final_pct['prediction_points'] = prediction_points
final_pct

,Team,Positive,Neutral,Negative,Points,Week,prediction_class,prediction_points
0,manutd,0.300564,0.316719,0.382716,1,1,Loss,0
1,LUFC,0.239769,0.261220,0.499011,1,1,Loss,0
2,BrentfordFC,0.400475,0.294318,0.305207,0,1,Win,3
3,everton,0.536857,0.204096,0.259047,3,1,Win,3
4,fulhamfc,0.428554,0.220831,0.350615,0,1,Win,3
...,...,...,...,...,...,...,...,...
119,lcfc,0.456937,0.272211,0.270851,3,8,Win,3
120,NFFC,0.368261,0.298210,0.333530,0,8,Win,3
121,CPFC,0.204617,0.542945,0.252438,1,8,Draw,1
122,SpursOfficial,0.172589,0.282557,0.544854,0,8,Loss,0


In [623]:
final_pct.to_csv('final_pct.csv')

In [624]:
score,prediction_class, prediction_points = evaluate(final_ttl)
final_ttl['prediction_class'] = prediction_class
final_ttl['prediction_points'] = prediction_points
final_ttl

,Team,Positive,Neutral,Negative,Points,Week,prediction_class,prediction_points
0,manutd,0.250000,0.250000,0.500000,1,1,Loss,0
1,LUFC,0.254902,0.254902,0.490196,1,1,Loss,0
2,BrentfordFC,0.428571,0.285714,0.285714,0,1,Win,3
3,everton,0.625000,0.125000,0.250000,3,1,Win,3
4,fulhamfc,0.500000,0.200000,0.300000,0,1,Win,3
...,...,...,...,...,...,...,...,...
119,lcfc,0.529412,0.235294,0.235294,3,8,Win,3
120,NFFC,0.456522,0.239130,0.304348,0,8,Win,3
121,CPFC,0.125000,0.750000,0.125000,1,8,Draw,1
122,SpursOfficial,0.111111,0.222222,0.666667,0,8,Loss,0


In [625]:
final_ttl.to_csv('final_ttl.csv')

Trying to use a pure prediction model below. Need more and cleaner data.

In [273]:
from keras.utils import to_categorical
y_train_onehot = to_categorical(label, num_classes=3)

In [274]:
y_train_onehot

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [566]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
#def remove_stop_words(text):
#    text = word_tokenize(text.lower())
#    stop_words = set(stopwords.words('english'))
#    text = [word for word in text if word.isalpha() and not word in stop_words]
#    return ' '.join(text)
     
x = final.apply(lambda row: row['Tweet'], axis=1)
y = y_train_onehot
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
 
max_words = 2000
max_length = 30
 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
x = pad_sequences(sequences, maxlen=max_length)

In [567]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, SimpleRNN
from keras.layers import Embedding
from keras.layers import LeakyReLU
from keras.optimizers import SGD

optimizer = SGD(lr=0.01)
model = Sequential() 
model.add(Embedding(max_words, 32, input_length=max_length)) 
model.add(SimpleRNN(32))
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(128))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
model.summary()

Model: "sequential_107"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_107 (Embedding)   (None, 30, 32)            64000     
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, 32)                2080      
                                                                 
 dense_240 (Dense)           (None, 128)               4224      
                                                                 
 leaky_re_lu_35 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_241 (Dense)           (None, 128)               16512     
                                                                 
 dense_242 (Dense)           (None, 3)                 387       
                                                                 
Total params: 87,203
Trainable params: 87,203
Non-tr

In [ ]:
hist = model.fit(x, y, validation_split=0.35, epochs=80, batch_size=64)

Epoch 1/5000
33/33 [==============================] - 3s 36ms/step - loss: 1.0818 - accuracy: 0.4295 - val_loss: 1.0750 - val_accuracy: 0.4518
Epoch 2/5000
33/33 [==============================] - 1s 17ms/step - loss: 1.0749 - accuracy: 0.4338 - val_loss: 1.0744 - val_accuracy: 0.4518
Epoch 3/5000
33/33 [==============================] - 1s 16ms/step - loss: 1.0725 - accuracy: 0.4343 - val_loss: 1.0656 - val_accuracy: 0.4518
Epoch 4/5000
33/33 [==============================] - 1s 16ms/step - loss: 1.0705 - accuracy: 0.4343 - val_loss: 1.0823 - val_accuracy: 0.4527
Epoch 5/5000
33/33 [==============================] - 1s 18ms/step - loss: 1.0705 - accuracy: 0.4338 - val_loss: 1.0871 - val_accuracy: 0.4527
Epoch 6/5000
33/33 [==============================] - 1s 20ms/step - loss: 1.0684 - accuracy: 0.4348 - val_loss: 1.0528 - val_accuracy: 0.4401
Epoch 7/5000
33/33 [==============================] - 1s 30ms/step - loss: 1.0677 - accuracy: 0.4377 - val_loss: 1.0721 - val_accuracy: 0.4527

In [557]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
 
acc = hist.history['accuracy']
val = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, '-', label='Training accuracy')
plt.plot(epochs, val, ':', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()

In [59]:
for matchweek in os.listdir('Proj_Data\\data\\Matchweeks'): #where the data is stored
    if matchweek == '.ipynb_checkpoints': #only want the data
        continue
    df = pd.read_csv('Proj_Data\\data\\Matchweeks\\' + matchweek)
    df = df.iloc[:,1:6].copy()
    week = int(matchweek[0])-1
    if int(matchweek[0])-1 == 1:
        final = df.copy()
        continue
    final = pd.concat([final,df]).reset_index(drop = True)